In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import arviz as az
import pymc as pm
import matplotlib.pyplot as plt

In [9]:
PATH = 'data/New_DB.xlsx'
df = pd.read_excel(PATH)

columns =['slope', 'PP', 'valor_humedad_suelo1']
X = df.loc[:,columns]
train_size = 0.8
df_general = X * 1
n_df_general = X * 0.85
df_general['output'] = 1
n_df_general['output'] = 0

df_output = pd.concat([df_general, n_df_general])

X_slope_PP = df_output.drop('valor_humedad_suelo1', axis=1)
X_slope_PP_vhs1 = df_output

X_slope_PP = X_slope_PP.dropna()
X_slope_PP_vhs1 = X_slope_PP_vhs1.dropna()
X_slope_PP = X_slope_PP.reset_index(drop=True)
X_slope_PP_vhs1 = X_slope_PP_vhs1.reset_index(drop=True)

y_slope_PP = X_slope_PP.output
X_slope_PP = X_slope_PP.drop(labels='output', axis=1)

y_slope_PP_vhs1 = X_slope_PP_vhs1.output
X_slope_PP_vhs1 = X_slope_PP_vhs1.drop(labels='output', axis=1)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pytensor
import pytensor.tensor as pt

In [10]:
#X_slope_PP_vhs1['valor_pp'] = X_slope_PP_vhs1['PP'] / X_slope_PP_vhs1['valor_humedad_suelo1']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_slope_PP_vhs1, y_slope_PP_vhs1, train_size=train_size, stratify=y_slope_PP_vhs1, random_state=42)
sc = StandardScaler()
floatX = pytensor.config.floatX
X_slope_PP_vhs1 = X_slope_PP_vhs1.astype(floatX)
y_slope_PP_vhs1 = y_slope_PP_vhs1.astype(floatX)
X_train, X_test, y_train, y_test = train_test_split(X_slope_PP_vhs1, y_slope_PP_vhs1, train_size=train_size, stratify=y_slope_PP_vhs1, random_state=42)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

random_seed = 12
rng = np.random.default_rng(random_seed)

In [246]:
def construct_nn(ann_input, ann_output):
    n_hidden = 5

    # Initialize random weights between each layer
    init_1 = rng.standard_normal(size=(X_train.shape[1], n_hidden)).astype(floatX)
    init_2 = rng.standard_normal(size=(n_hidden, n_hidden)).astype(floatX)
    init_out = rng.standard_normal(size=n_hidden).astype(floatX)

    coords = {
        "hidden_layer_1": np.arange(n_hidden),
        "hidden_layer_2": np.arange(n_hidden),
        "train_cols": np.arange(X_train.shape[1]),
        # "obs_id": np.arange(X_train.shape[0]),
    }
    with pm.Model(coords=coords) as neural_network:
        ann_input = pm.Data("ann_input", X_train, mutable=True, dims=("obs_id", "train_cols"))
        ann_output = pm.Data("ann_output", y_train, mutable=True, dims="obs_id")

        # Weights from input to hidden layer
        weights_in_1 = pm.Normal(
            "w_in_1", 0, sigma=1, initval=init_1, dims=("train_cols", "hidden_layer_1")
        )

        # Weights from 1st to 2nd layer
        weights_1_2 = pm.Normal(
            "w_1_2", 0, sigma=1, initval=init_2, dims=("hidden_layer_1", "hidden_layer_2")
        )

        # Weights from hidden layer to output
        weights_2_out = pm.Normal("w_2_out", 0, sigma=1, initval=init_out, dims="hidden_layer_2")

        # Build neural-network using tanh activation function
        act_1 = pm.math.tanh(pm.math.dot(ann_input, weights_in_1))
        act_2 = pm.math.tanh(pm.math.dot(act_1, weights_1_2))
        act_out = pm.math.sigmoid(pm.math.dot(act_2, weights_2_out))

        # Binary classification -> Bernoulli likelihood
        out = pm.Bernoulli(
            "out",
            act_out,
            observed=ann_output,
            total_size=y_train.shape[0],  # IMPORTANT for minibatches
            dims="obs_id",
        )
    return neural_network


neural_network = construct_nn(X_train, y_train)

In [247]:
%%time
with neural_network:
    approx = pm.fit(n=30_000)

Interrupted at 3,498 [11%]: Average Loss = 362.64


CPU times: total: 20.1 s
Wall time: 5.1 s


In [ ]:
trace = approx.sample(draws=5000)
with neural_network:
    pm.set_data(new_data={"ann_input": X_test})
    ppc = pm.sample_posterior_predictive(trace)
    trace.extend(ppc)

pred = ppc.posterior_predictive["out"].mean(("chain", "draw")) > 0.50
print(f"Accuracy = {(y_test == pred.values).mean() * 100}%")

Sampling: [out]


Accuracy = 91.96428571428571%


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [ ]:
"""X_train, X_test, y_train, y_test = train_test_split(X_slope_PP_vhs1, y_slope_PP_vhs1, train_size=train_size, stratify=y_slope_PP_vhs1, random_state=random_seed)

slope_mean = X_train['slope'].mean()
slope_std = X_train['slope'].std()

PP_mean = X_train['PP'].mean()
PP_std = X_train['PP'].std()

vhs1_mean = X_train['valor_humedad_suelo1'].mean()
vhs1_std = X_train['valor_humedad_suelo1'].std()

X_train['slope'] = (X_train['slope'] - slope_mean) / slope_std
X_test['slope'] = (X_test['slope'] - slope_mean) / slope_std

X_train['PP'] = (X_train['PP'] - PP_mean) / PP_std
X_test['PP'] = (X_test['PP'] - PP_mean) / PP_std

X_train['valor_humedad_suelo1'] = (X_train['valor_humedad_suelo1'] - vhs1_mean) / vhs1_std
X_test['valor_humedad_suelo1'] = (X_test['valor_humedad_suelo1'] - vhs1_mean) / vhs1_std
"""

"X_train, X_test, y_train, y_test = train_test_split(X_slope_PP_vhs1, y_slope_PP_vhs1, train_size=train_size, stratify=y_slope_PP_vhs1, random_state=random_seed)\n\nslope_mean = X_train['slope'].mean()\nslope_std = X_train['slope'].std()\n\nPP_mean = X_train['PP'].mean()\nPP_std = X_train['PP'].std()\n\nvhs1_mean = X_train['valor_humedad_suelo1'].mean()\nvhs1_std = X_train['valor_humedad_suelo1'].std()\n\nX_train['slope'] = (X_train['slope'] - slope_mean) / slope_std\nX_test['slope'] = (X_test['slope'] - slope_mean) / slope_std\n\nX_train['PP'] = (X_train['PP'] - PP_mean) / PP_std\nX_test['PP'] = (X_test['PP'] - PP_mean) / PP_std\n\nX_train['valor_humedad_suelo1'] = (X_train['valor_humedad_suelo1'] - vhs1_mean) / vhs1_std\nX_test['valor_humedad_suelo1'] = (X_test['valor_humedad_suelo1'] - vhs1_mean) / vhs1_std\n"

In [ ]:
"""from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

def augment_data_with_knn(data, model, num_neighbors=5, num_augmentations=2):
    augmented_data = []

    for _, row in data.iterrows():
        features = row[['slope', 'PP', 'valor_humedad_suelo1', 'valor_pp']].values.reshape(1, -1)
        neighbors = model.kneighbors(features, n_neighbors=num_neighbors, return_distance=False)[0]

        for _ in range(num_augmentations):
            # Selecciona aleatoriamente uno de los vecinos más cercanos
            neighbor_index = np.random.choice(neighbors)
            augmented_row = X_train.iloc[neighbor_index].to_dict()
            augmented_row['output'] = y_train.iloc[neighbor_index]
            augmented_data.append(augmented_row)

    return pd.DataFrame(augmented_data)

augemented_df = augment_data_with_knn(X_test, knn_model, num_neighbors=5)

augemented_df.head() , len(augemented_df)"""

"from sklearn.neighbors import KNeighborsClassifier\n\nknn_model = KNeighborsClassifier(n_neighbors=5)\nknn_model.fit(X_train, y_train)\n\ndef augment_data_with_knn(data, model, num_neighbors=5, num_augmentations=2):\n    augmented_data = []\n\n    for _, row in data.iterrows():\n        features = row[['slope', 'PP', 'valor_humedad_suelo1', 'valor_pp']].values.reshape(1, -1)\n        neighbors = model.kneighbors(features, n_neighbors=num_neighbors, return_distance=False)[0]\n\n        for _ in range(num_augmentations):\n            # Selecciona aleatoriamente uno de los vecinos más cercanos\n            neighbor_index = np.random.choice(neighbors)\n            augmented_row = X_train.iloc[neighbor_index].to_dict()\n            augmented_row['output'] = y_train.iloc[neighbor_index]\n            augmented_data.append(augmented_row)\n\n    return pd.DataFrame(augmented_data)\n\naugemented_df = augment_data_with_knn(X_test, knn_model, num_neighbors=5)\n\naugemented_df.head() , len(augemen

In [ ]:
"""y_train = pd.concat([y_train, augemented_df.output])
X_train = pd.concat([X_train, augemented_df.drop(labels='output', axis=1)])
X_train.shape, X_test.shape
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)"""

"y_train = pd.concat([y_train, augemented_df.output])\nX_train = pd.concat([X_train, augemented_df.drop(labels='output', axis=1)])\nX_train.shape, X_test.shape\nsc = StandardScaler()\nX_train = sc.fit_transform(X_train)\nX_test = sc.transform(X_test)"

In [ ]:
neural_network = construct_nn(X_train, y_train)
with neural_network:
    approx = pm.fit(n=30_000)

trace = approx.sample(draws=5000)
with neural_network:
    pm.set_data(new_data={"ann_input": X_test})
    ppc = pm.sample_posterior_predictive(trace)
    trace.extend(ppc)

pred = ppc.posterior_predictive["out"].mean(("chain", "draw")) > 0.50
print(f"Accuracy = {(y_test == pred.values).mean() * 100}%")

Finished [100%]: Average Loss = 183.32
Sampling: [out]


Accuracy = 91.07142857142857%


In [18]:
def bayesian_regression_tree(input, output):
    coords = {
        'train_cols': np.arange(X_train.shape[1]),
        #'obs_id': np.arange(X_train.shape[0])
    }
    with pm.Model(coords=coords) as model:
        input = pm.Data('input', X_train, mutable=True, dims=('obs_id','train_cols'))
        observed = pm.Data('observed', y_train, mutable=True, dims='dims_id')
        intercept = pm.Normal('intercept', mu = 0, sigma = 10) #5
        beta = pm.Normal('beta',mu=0, sigma=5)
        beta2 = pm.Normal('beta2', mu=0, sigma=5)
        beta3 = pm.Normal('beta3', mu = 0, sigma=5)
        alpha = pm.Normal('alpha', mu = 0, sigma=0.1)
        alpha2 = pm.Normal('alpha2', mu = 0, sigma = 0.1) #0.1
        alpha3 = pm.Normal('alpha3', mu = 0, sigma=1)
        alpha4 = pm.Normal('alpha4', mu = 0, sigma = 0.1) #0.1
        alpha5 = pm.Normal('alpha5', mu = 0 , sigma = 1) #1
        alpha6 = pm.Normal('alpha6', mu = 0 , sigma = 1) #1
        p = pm.Deterministic(
            'p', 
            pm.math.invlogit(
                intercept + input[:,0]*beta + input[:,1]*beta2 + input[:,2]*beta3 + 
                (input[:,1]**2)*alpha**2 + (input[:,2]**2)*alpha2**2 + (input[:,0]**2)*alpha3**2 + 
                (input[:,1]**3)*alpha4**3 + (input[:,2]**3)*alpha5**3 + (input[:,0]**3)*alpha6**3))
        final = pm.Bernoulli('out',p , observed = observed, dims='obs_id')
        
    return model

bayesian_regression_tree = bayesian_regression_tree(X_train, y_train)

In [19]:
with bayesian_regression_tree:
    start = pm.find_MAP()
    #hessian_inv = np.linalg.inv(pm.find_hessian(start, model=bayesian_regression_tree))
    #approx = pm.fit(n=10_000, start=start)
    step = pm.Metropolis()
    trace = pm.sample(draws=5000, step = step, start = start)# nuts_sampler_kwargs={'hess_inv': hessian_inv})

with bayesian_regression_tree:
    pm.set_data(new_data={"input": X_test})
    ppc = pm.sample_posterior_predictive(trace, var_names=['p'], return_inferencedata=True, predictions=True, extend_inferencedata=True)
    trace.extend(ppc)
#pred = ppc.posterior_predictive["out"].mean(("chain", "draw")) > 0.50
#print(f"Accuracy = {(y_test == pred.values).mean() * 100}%")
pred = ppc.predictions['p'].mean(('chain', 'draw')) > 0.5
print(f"Accuracy = {(y_test == pred.values).mean() * 100}%")

C:\Users\Manuel\AppData\Local\Temp\ipykernel_14996\2638288232.py:6: FutureWarning: The `start` kwarg was renamed to `initvals` and can now do more. Please check the docstring.
  trace = pm.sample(draws=5000, step = step, start = start)# nuts_sampler_kwargs={'hess_inv': hessian_inv})
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [intercept]
>Metropolis: [beta]
>Metropolis: [beta2]
>Metropolis: [beta3]
>Metropolis: [alpha]
>Metropolis: [alpha2]
>Metropolis: [alpha3]
>Metropolis: [alpha4]
>Metropolis: [alpha5]
>Metropolis: [alpha6]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 49 seconds.
Sampling: []


Accuracy = 88.39285714285714%
